In [1]:
from edge_analyzer.acquisition import airt
from edge_analyzer import channel_waveform

duration = 0.5
sample_rate_Hz = 15.36e6
analysis_bandwidth_Hz = 10e6

print('connecting...')
capture = airt.AirTCapture(frequency_Hz=2.44e9, sample_rate_Hz=sample_rate_Hz)

try:
    print('acquiring...')
    raw_iq = capture.acquire(int(duration*sample_rate_Hz))
finally:
    print('done')
    capture.close()

connecting...
acquiring...
done


In [2]:
# import importlib
# from edge_analyzer import channel_waveform
# channel_waveform = importlib.reload(channel_waveform)

analysis_spec = {
    'power_time_series': {'detector_period': 10e-3, 'detectors': ('rms', 'peak')},
    'cyclic_channel_power': {
        'cyclic_period': 10e-3,
        'detector_period': 1e-3 / 15 / 4,
        'detectors': ('rms', 'peak'),
        'cyclic_statistics': ('min', 'mean', 'max'),
    },
    'persistence_spectrum': {
        'window': 'flattop',
        'resolution': 15e3,
        'quantiles': [0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99, 0.999, 1],
        'truncate': True,
        'dB': True
    },
    'amplitude_probability_distribution': {
        'power_low': -40,
        'power_high': 15,
        'power_count': 221,  # 0.25 dB resolution
    },    
}

filter_spec = {
    # 'iir': {
    #     'passband_ripple_dB': 0.1,
    #     'stopband_attenuation_dB': 90,
    #     'transition_bandwidth_Hz': 250e3,
    # },
    'ola': {
        'noverlap': 1024,
        'window': 'hamming' # 'hamming', 'blackman', or 'blackmanharris'
    },
}

channel_waveform.from_spec(
    raw_iq,
    sample_rate_Hz,
    analysis_bandwidth_Hz=analysis_bandwidth_Hz,
    filter_spec=filter_spec,
    analysis_spec=analysis_spec,
)

<xarray.Dataset> Size: 77kB
Dimensions:                             (power_detector: 2, time_elapsed: 50,
                                         persistence_statistic: 9,
                                         baseband_frequency: 667,
                                         channel_power: 221,
                                         cyclic_statistic: 3, cyclic_lag: 600)
Coordinates:
  * power_detector                      (power_detector) <U4 32B 'rms' 'peak'
  * time_elapsed                        (time_elapsed) float64 400B 0.0 ... 0.49
  * persistence_statistic               (persistence_statistic) <U5 180B '0.1...
  * baseband_frequency                  (baseband_frequency) float64 5kB -4.9...
  * channel_power                       (channel_power) float64 2kB -40.0 ......
  * cyclic_statistic                    (cyclic_statistic) <U4 48B 'min' ... ...
  * cyclic_lag                          (cyclic_lag) float64 5kB 0.0 ... 0.00...
Data variables:
    power_time_series                   (power_detector, time_elapsed) float32 400B ...
    persistence_spectrum                (persistence_statistic, baseband_frequency) float64 48kB ...
    amplitude_probability_distribution  (channel_power) float64 2kB 0.0 ... 0.0
    cyclic_channel_power                (power_detector, cyclic_statistic, cyclic_lag) float32 14kB ...
Attributes:
    sample_rate_Hz:         15360000.0
    analysis_bandwidth_Hz:  10000000.0
    filter_specification:   {}

In [3]:
from cupyx.profiler import benchmark


benchmark(
    channel_waveform.from_spec,
    kwargs=dict(
        iq=raw_iq,
        sample_rate_Hz=sample_rate_Hz,
        analysis_bandwidth_Hz=analysis_bandwidth_Hz,
        filter_spec=filter_spec,
        analysis_spec=analysis_spec,
    ),
    n_repeat=5
)

from_spec           :    CPU: 509669.934 us   +/- 4414.902 (min: 506477.467 / max: 518137.325) us     GPU-0: 509736.572 us   +/- 4260.377 (min: 506620.483 / max: 517846.375) us